# Get data stored in Google drive
Don't run these 2 lines if one loads data from computer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Import neccesary modules

In [ ]:
import os
import time
import numpy as np
from PIL import Image  
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Conv2D, Dropout, MaxPool2D, BatchNormalization
from tensorflow.keras import optimizers
from keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.losses import CategoricalCrossentropy

# Load data

In [ ]:
def load_training_image(dir):
    """
    Load training data in image format (and save in it .npy format for later use)
    
    :param dir: Directory storing data
    :return X_shuffle, y_shuffle: Shuffled training data and theirs one-hot encoded labels
    """

    LABEL_LIST = [10, 1, 2, 3, 4, 5, 6, 7, 8, 9]
    
    X = []
    y_onehot = np.zeros((60000, 10))
    
    for num in range(10):
        sub_dir = os.path.join(dir, str(LABEL_LIST[num]))
        for img_dir in os.listdir(sub_dir):
            img = Image.open(os.path.join(sub_dir, img_dir))
            array_img = np.array(img)
            X.append(array_img)
        
        y_onehot[6000*(num):6000*(num+1), num] = 1  # Store label 10 as label 0
    
        # Show example of number
        # img.show(title=str(num+1))
        # plt.figure()
        # plt.title(str(LABEL_LIST[num]))
        # plt.imshow(array_img)
        # plt.show()
    
    X = np.reshape(np.array(X), (60000, 28, 28, 1))
    X_shuffle, y_shuffle = shuffle(X, y_onehot, random_state=1)
    
    # Save data and theirs label to .npy format
    np.save('data.npy', X_shuffle)
    np.save('label.npy', y_shuffle)
    
    return X_shuffle, y_shuffle

def load_test_image(dir):
    """
    Load test data in image format (and save in it .npy format for later use)
    
    :param dir: Directory storing data
    :return X: Test data 
    """

    X = []

    for img_dir in os.listdir(dir):
        img = Image.open(os.path.join(dir, img_dir))
        X.append(np.array(img))

    X = np.reshape(np.array(X), (len(X), 28, 28, 1))
    np.save("test.npy", X)

    return X

In [ ]:
training_folder = "/content/drive/My Drive/Colab Notebooks/training_data" 
X, y = load_training_image(my_folder)

test_folder = "/content/drive/My Drive/Colab Notebooks/test"
X_test = load_test_image(test_folder)

In [ ]:
X, y = np.load("data.npy"), np.load("data.npy")
X_test = np.load("test.npy")

# Splitting dataset
One can skip this part parameter "validation_split" is used in method fit() of tensorflow.keras.models.

In [ ]:
X_train = X[:50000, :, :] 
y_train = y[:50000, :] 

X_small = X_train[:1000, :, :]
y_small = y[:1000, :] 

X_test = X[50000:, :, :] 
y_test = y[50000:, :] 

# Create model: CNN

In [ ]:
model_cnn = Sequential([
    Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    BatchNormalization(),

    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    BatchNormalization(),

    Conv2D(256, (3, 3), activation='relu', padding='same'),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    BatchNormalization(),
        
    Flatten(),
    Dense(512, activation='relu'),
    Dense(10, activation='softmax', activity_regularizer=l2(1e-4))
    ])

opt = optimizers.Adam(learning_rate=1e-3)
model_cnn.compile(optimizer=opt, loss=CategoricalCrossentropy(), metrics=['accuracy'])
model_cnn.summary()


# Fit model with training data

In [ ]:
history = model_cnn.fit(X, y, validation_split=0.2, epochs=200, verbose=1)

# Plot accuracy and loss

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(15, 5))

ax[0].plot(history.history['accuracy'], label="Training accuracy")
ax[0].plot(history.history['val_accuracy'], label="Validation accuracy")
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')
ax[0].set_ylim((0.99, 1.0))
ax[0].legend(loc='lower right')
ax[0].grid("on")

ax[1].plot(history.history['loss'], label="Training loss")
ax[1].plot(history.history['val_loss'], label="Validation loss")
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')
ax[1].set_ylim((0.0, 0.05))
ax[1].legend(loc='upper right')
ax[1].grid("on")

fig.tight_layout()
plt.show()

# Test

In [ ]:
y_prob = model_cnn.predict(X_test)
y_eval = np.argmax(y_prob, axis=1)

# Change label 0 to 10
y_eval[y_eval == 0] = 10

# Save result to .csv file
with open("sample_submission.csv", "w") as fp: 
    fp.write("Id,Category\n") 
    for idx in range(10000): 
        fp.write(f"{idx:05},{y_eval[idx]}\n") 

# Saving model

In [ ]:
model_cnn.save("cnn_mode_l.h5")

# Loading model to run on test set

In [ ]:
from tensorflow.keras.models import load_model
new_model = load_model("cnn_mode_l.h5")    # Can only save architechture, need to find a better way to save weight
new_model.summary()